Helper functions to download trading data

In [1]:
# Import packages
import pandas
import time
import requests
from datetime import datetime, timedelta

# GDAX download functions

def date_to_iso8601(date):
    return '{year}-{month:02d}-{day:02d}T{hour:02d}:{minute:02d}:{second:02d}'.format(
      year=date.year,
      month=date.month,
      day=date.day,
      hour=date.hour,
      minute=date.minute,
      second=date.second)


def gdax_request_trade_slice(product_id, start, end, granularity):
    """
    Single HTTP request function with error catching and management for server error responses
    Response is in the format: [[time, low, high, open, close, volume], ...]
    """
    # Allow 3 retries (we might get rate limited).
    retries = 3
    
    # Change dates to iso8601 format as specified
    iso_start = date_to_iso8601(start)
    iso_end = date_to_iso8601(end)

    # Set uri
    uri = 'https://api.gdax.com/products/{currency_pair}/candles'.format(currency_pair=product_id)
    
    for retry_count in range(0, retries):
        response = requests.get(uri, {
          'start': iso_start,
          'end': iso_end,
          'granularity': granularity * 60 # Converting to seconds for API
        })
        if response.status_code != 200:
            if retry_count + 1 == retries:
                raise Exception('Failed to get exchange data for ({}, {})! Error message: {}'.format(start, end, response.text))
            else:
                # Exponential back-off.
                time.sleep(1.5 ** retry_count)
        else:
            # Sort the historic rates (in ascending order) based on the timestamp.
            result = sorted(response.json(), key=lambda x: x[0])
            return result

        
def gdax_request_order_book(product_id, level):
    """
    Returns the current order book for a currency pair. 
    :Level: Granularity level 1, 2, or 3 as defined by API. Default = 1, preferred for my usage is 2.
    Level 2 response example: 
    {
        "sequence": "3",
        "bids": [
            [ price, size, num-orders ],
            [ "295.96", "4.39088265", 2 ],
            ...
        ],
        "asks": [
            [ price, size, num-orders ],
            [ "295.97", "25.23542881", 12 ],
            ...
        ]
    }
    """
    # Allow 3 retries (we might get rate limited).
    retries = 3

    # Set uri
    uri = 'https://api.gdax.com/products/{currency_pair}/book'.format(currency_pair=product_id)
    
    for retry_count in range(0, retries):
        response = requests.get(uri, {
          'level': level
        })
        if response.status_code != 200 or not len(response.json()):
            if retry_count + 1 == retries:
                raise Exception('Failed to get order book data! Error message: {}'.format(response.text))
            else:
                # Exponential back-off.
                time.sleep(1.5 ** retry_count)
        else:
            return result

        
def gdax_trade_downloader(currency_pair, start, end, interval):
    """
    Breaks up gdax trade data requests into chunks of 200 candlesticks to download in 1 second intervals, to comply with GDAX API rules
    :currency_pair: string with requested crypto-fiat pair
    :start: start of time period as datetime object
    :end: end of time period as datetime object
    :interval: candlestick intervals in ninutes
    Returns an array with rows of candlestick data in the following format: [timestamp, low, high, open, close, volume]    
    """      
    data = [] # Empty list to append data 
    delta = timedelta(minutes=interval * 200) # 200 intervals per request
    slice_start = start
    while slice_start != end:
        slice_end = min(slice_start + delta, end)
        print("downloading {} data from {} to {}".format(currency_pair, slice_start, slice_end))
        data += gdax_request_trade_slice(
                product_id=currency_pair,
                start=slice_start,
                end=slice_end,
                granularity=interval
        )
        slice_start = slice_end
        time.sleep(0.5)

    data_frame = pandas.DataFrame(data=data, columns=['time', 'low', 'high', 'open', 'close', 'volume'])
    data_frame.set_index('time', inplace=True)
    return data_frame


def gdax_order_book_downloader(currency_pair, interval, start_time, end_time):
    """
    Script to download order book at regular intervals. Will work out how to use this data with deep learning network later :)
    :currency_pair: string with requested crypto-fiat pair
    :interval: request intervals in minutes
    :start_time: datetime object - the time at which this operation should start requesting data
    :end_time: datetime object - the time at which this operation should finish
    Returns something...
    """
    data = [] # Empty list to append data
    
    # Calculate time to next interval
    time_to_start =  min(start_time - datetime.datetime.now(),0) # Start when specified, or now if specified time is in the past
    time.sleep(time_to_start)
    
    while datetime.datetime.now() < end_time:
        data += gdax_request_order_book(
                product_id=currency_pair,
                level=2
        )
        time.sleep(interval*60) # sleep takes time in seconds
    
    return data
    
    # Need to work out what I want to do with this. Order book has 2 pieces of important information. Presence of any walls, and overall size    
    
    

In [32]:
# Kraken API
import numpy as np
import krakenex
import pytz
from pykrakenapi import KrakenAPI

def datetime_to_interval(dt, interval): # Round a datetime DOWN to the nearest interval
    interval = datetime(dt.year, dt.month, dt.day, dt.hour, dt.minute // interval * interval)
    return interval

def kraken_request_trade_slice(currency_pair, start, interval):    
    """
    Initializes Kraken API from pykrakenapi library and calls get_trade function. Batches trade data into OHLC (candles) 
    :currency_pair: string with requested crypto-fiat pair
    :start: start of time period as datetime object, defaults to earliest if not passed through
    :interval: candlestick intervals in ninutes
    Returns an array with rows of candlestick data in the following format: ['time', 'low', 'high', 'open', 'close', 'volume']    
    """     
    api = krakenex.API()
    k = KrakenAPI(api)
    timestamp = int(start.replace(tzinfo=pytz.utc).timestamp())*1000000000
    trades, last = k.get_recent_trades(currency_pair, since=timestamp, ascending=True)    
    return trades, last/1000000000

def kraken_to_ohlc(trades):
    """
    Groups and processes individual trade data to return OHLC (candle) data
    :trades: trades pandas dataframe as returned by kraken_trade_downloader
    Returns an array with rows of candlestick data in the following format: ['time', 'low', 'high', 'open', 'close', 'volume']    
    """
    # Converts unix timestamp data into candle interval periods. Period time corresponds to beginning of period
    trades['datetime'] = pandas.to_datetime(trades['time'], unit='s') # Reformat unix timestamp as datetime
    trades['period'] = trades['datetime'].map(lambda x: datetime_to_interval(x, interval))
    
    # Group trades into periods to aggregate volume and get low/high price
    trade_agg = trades.groupby('period')
    trade_agg = trade_agg.agg({
        'price': {'low': np.min, 'high': np.max},
        'volume': np.sum
    })
    
    # Create a fresh pandas dataframe and copies data from aggregated trades dataframe above
    ohlc = pandas.DataFrame(index=trade_agg.index.values, columns=['low', 'high', 'open', 'close', 'volume'])
    ohlc['low'] = trade_agg['price']['low']
    ohlc['high'] = trade_agg['price']['high']
    ohlc['volume'] = trade_agg['volume']['sum']
    
    # Iterate through each candle period and searches original trades dataframe for the first and last price, then set that
    for i, row in ohlc.iterrows():
        selection = trades.loc[trades['period'] == i] # Return all rows for the current period
        first_trade_index = selection['time'].idxmax()
        first_trade_price = selection.loc[first_trade_index]['price']
        ohlc.at[i, 'open'] = first_trade_price
        last_trade_index = selection['time'].idxmin()
        last_trade_price = selection.loc[last_trade_index]['price']
        ohlc.at[i, 'close'] = last_trade_price

    return ohlc



In [33]:
interval = 5
currency_pair = 'XETHZUSD'
start = datetime(2016,10,10)
end = datetime(2016,10,14)

def kraken_trade_downloader(currency_pair, start, end, interval):
    slice_start = start
    end_timestamp = end.replace(tzinfo=pytz.utc).timestamp()
    data, last = kraken_request_trade_slice(currency_pair, slice_start, interval)
    while last < end_timestamp:        
        slice_start = datetime.fromtimestamp(last)
        new_data, last = kraken_request_trade_slice(currency_pair, slice_start, interval)
        data = data.append(new_data)
        time.sleep(1)
        print("time period from {} to {}".format(slice_start, last))
    ohlc = kraken_to_ohlc(data)
    return ohlc

ohlc = kraken_trade_downloader(currency_pair, start, end, interval)

time period from 2016-10-11 03:10:50.489375 to 1476285267.537371
time period from 2016-10-12 08:14:27.537371 to 1476374377.2924552
time period from 2016-10-13 08:59:37.292455 to 1476469666.8383987


C:\Users\alast\Anaconda3\envs\traderbot\lib\site-packages\pandas\core\groupby.py:4281: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [35]:
start = datetime(2015,5,1)
end = datetime(2015,5,2)

# Get GDAX data
gdax_pairs = ['BTC-USD']

for pair in gdax_pairs:    
    data = gdax_trade_downloader(pair, start, end, 5)

    print(data)
# Get Kraken data
#kraken_pairs = ['XETHZUSD',
               #'XETHZEUR',
               #'XXBTZUSD',
               #'XXBTZEUR',
               #'XLTCZUSD',
               #'XLTCZEUR']

#for pair in kraken_pairs:
#    print(pair)
#    data, last = kraken_trade_downloader(pair, start, 5)
#    print(data.head())

    

downloading BTC-USD data from 2015-05-01 00:00:00 to 2015-05-01 16:40:00
downloading BTC-USD data from 2015-05-01 16:40:00 to 2015-05-02 00:00:00
               low    high    open   close     volume
time                                                 
1430438400  236.42  236.78  236.78  236.50   1.626472
1430438700  236.01  236.49  236.49  236.01   2.784245
1430439000  236.00  236.10  236.01  236.00  73.308585
1430439300  236.00  236.04  236.04  236.04   6.197529
1430439600  235.79  236.00  236.00  235.80  15.201524
1430439900  235.73  235.82  235.80  235.74  13.571108
1430440200  235.74  235.78  235.74  235.74   4.429600
1430440500  235.74  235.80  235.75  235.77   1.208400
1430440800  235.68  235.83  235.76  235.71   5.973491
1430441100  235.75  235.92  235.75  235.90   0.107201
1430441400  235.91  236.21  235.92  236.21  22.815325
1430441700  236.27  236.40  236.28  236.40   1.700810
1430442000  236.42  238.09  236.42  236.99   5.950986
1430442300  236.96  237.19  236.96  237.18  

Helper functions to download sentiment data

Data pre-processing

In [ ]:
# Calculate growth rates. Growth rate prediction is the target

Define RNN

In [ ]:
#Basic RNN first
#Then with multiple types of data, try a convnet + RNN

Train RNN

Define test function

In [4]:
data = gdax_request_trade_slice(
                product_id='ETH-USD',
                start=start,
                end=end,
                granularity=5)

print(data.head())

Exception: Failed to get exchange data for (2014-01-01 00:00:00, 2014-01-01 08:20:00)! Error message: []

In [11]:
start = datetime(2014,1,1,0,0)
end = datetime(2014,1,1,8,20)
iso_start = date_to_iso8601(start)
iso_end = date_to_iso8601(end)


data = requests.get('https://api.gdax.com/products/ETH-USD/candles', {'start': iso_start,
                                                                     'end': iso_end,
                                                                     'granularity': 5 * 60})
print(data.json())

[]


In [3]:
start = datetime(2014,1,1)
end = datetime(2017,12,31)
pair = 'ETH-USD'
start, end = test(pair, start, end, 5)
print(start, end)

2014-01-01 00:00:00 2014-01-01 08:20:00


In [1]:
start = datetime(2017,1,1,0,0)
end = datetime(2017,1,1,8,20)

data = gdax_request_trade_slice('ETH-USD', start, end, 5)

NameError: name 'datetime' is not defined

In [21]:
import pickle
try:
    with open('pickles/kraken_USD_data.pickle', 'rb') as f:
            K_ETH_USD, K_BTC_USD, K_LTC_USD = pickle.load(f)
except:
    print("No Kraken USD pickle found")
try:
    with open('pickles/kraken_EUR_data.pickle', 'rb') as f:
            K_ETH_EUR, K_BTC_EUR, K_LTC_EUR = pickle.load(f)
except:
    print("No Kraken EUR pickle found")
try:
    with open('pickles/gdax_USD_data.pickle', 'rb') as f:
            G_ETH_USD, G_BTC_USD, G_LTC_USD = pickle.load(f)
except:
    print("No GDAX USD pickle found")
try:
    with open('pickles/gdax_EUR_data.pickle', 'rb') as f:
            G_ETH_EUR, G_BTC_EUR, G_LTC_EUR = pickle.load(f)
except:
    print("No GDAX EUR pickle found")
try:
    with open('pickles/reddit_data.pickle', 'rb') as f:
            ETH_reddit, BTC_reddit, LTC_reddit = pickle.load(f)
except:
    print("No Reddit pickle found")
try:
    with open('pickles/google_data.pickle', 'rb') as f:
            ETH_trends, BTC_trends, LTC_trends = pickle.load(f)
except:
    print("No Google trends pickle found")
try:
    with open('pickles/blockchain_data.pickle', 'rb') as f:
            ETH_blockchain, BTC_blockchain, LTC_blockchain = pickle.load(f)
except:
    print("No Blockchain pickle found")
    


No Reddit pickle found


In [32]:
import numpy as np

datasets = [G_ETH_USD, G_BTC_USD, G_LTC_USD, 
            K_ETH_USD, K_BTC_USD, K_LTC_USD, 
            G_ETH_EUR, G_BTC_EUR, G_LTC_EUR, 
            K_ETH_EUR, K_BTC_EUR, K_LTC_EUR, 
            ETH_trends, BTC_trends, LTC_trends,
            ETH_blockchain, BTC_blockchain, LTC_blockchain
           ]

count=0
for name in datasets:
    npdata = np.array(name)
    try: 
        npdata.astype('float32')
        count+=1
    except:
        print(count)
        count+=1
        

3
10
15
16
17


In [49]:
erroneous_datasets = [K_ETH_USD, K_BTC_EUR]

In [63]:
npdata = np.array(K_BTC_EUR)
for ii in range(len(npdata)):
    #print(ii)
    npdata[ii].astype('float32')

ValueError: setting an array element with a sequence.

In [57]:
K_ETH_USD.ix[886]


low                                                 1293.29
high                                                   1386
open                                                1305.69
close     dtime
2018-01-15 00:32:55.490800    1374.39
20...
volume                                              3664.54
Name: 2018-01-15 00:00:00, dtype: object

In [56]:
K_BTC_EUR.ix[717]

low                                                 1886.33
high                                                   2055
open      dtime
2017-07-18 22:24:30.913200    2037.998
2...
close                                               1891.69
volume                                              2083.45
Name: 2017-07-18 00:00:00, dtype: object

In [68]:
from Preprocessing import kraken, gdax, reddit, google_search, blockchain_stat_importer
interval = 1440
start_time = datetime(2015,1,1)
end_time = datetime(2018,2,28)

K = kraken.Kraken(interval, start_time, end_time)
K_ETH_USD = K.get_training_data('XETHZUSD')
#K_BTC_USD = K.get_training_data('XXBTZUSD')
#K_LTC_USD = K.get_training_data('XLTCZUSD')

# Get Kraken EUR market data
#K_ETH_EUR = K.get_training_data('XETHZEUR')
#K_BTC_EUR = K.get_training_data('XXBTZEUR')
#K_LTC_EUR = K.get_training_data('XLTCZEUR')


time period from 2015-08-28 18:21:53.841708 to 1445544380.3089046
time period from 2015-10-23 03:06:20.308905 to 1447869962.80071
time period from 2015-11-19 01:06:02.800710 to 1451276017.040883
time period from 2015-12-28 11:13:37.040883 to 1453111969.4657853
time period from 2016-01-18 17:12:49.465785 to 1453585576.7740574
time period from 2016-01-24 04:46:16.774057 to 1453794531.7005613
time period from 2016-01-26 14:48:51.700561 to 1454254102.943958
time period from 2016-01-31 22:28:22.943958 to 1454872805.5778718
time period from 2016-02-08 02:20:05.577872 to 1455069615.7676845
time period from 2016-02-10 09:00:15.767684 to 1455184947.8942022
time period from 2016-02-11 17:02:27.894202 to 1455288422.2193737
time period from 2016-02-12 21:47:02.219374 to 1455387831.3499224
time period from 2016-02-14 01:23:51.349922 to 1455547381.3308268
time period from 2016-02-15 21:43:01.330827 to 1455693706.200016
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time p

time period from 2016-05-22 22:14:55.103601 to 1464041620.4193845
time period from 2016-05-24 05:13:40.419384 to 1464107457.5884533
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-05-24 23:30:57.588453 to 1464245517.4253566
time period from 2016-05-26 13:51:57.425357 to 1464330378.875453
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
attempt: 000 | ['EService:Unavailable']
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2016-05-27 13:26:18.875453 to 1464415044.0311797
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-05-28 12:57:24.031180 to 1464522004.3654375
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
attempt: 000 | ['EService:Unavailable']
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-05-29 18:40:04.365438 to 1464691528.7665892
call rate

time period from 2016-08-06 04:34:27.871983 to 1470545794.7287288
time period from 2016-08-07 11:56:34.728729 to 1470681191.5237358
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2016-08-09 01:33:11.523736 to 1470773302.1731865
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-08-10 03:08:22.173187 to 1470868513.8885555
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-08-11 05:35:13.888556 to 1470946434.4913948
time period from 2016-08-12 03:13:54.491395 to 1471024891.8920918
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-08-13 01:01:31.892092 to 1471226990.0617456
time period from 2016-08-15 09:09:50.061746 to 1471419723.764364
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2016-08-17 14:42:03.764364 to 1471707372.6112456
time period from 2016-08-20 22:3

time period from 2017-01-02 11:08:55.468921 to 1483455647.3488395
time period from 2017-01-03 22:00:47.348840 to 1483515154.6396391
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-01-04 14:32:34.639639 to 1483570321.6907444
time period from 2017-01-05 05:52:01.690744 to 1483609169.7594175
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-01-05 16:39:29.759418 to 1483669746.8192143
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-01-06 09:29:06.819214 to 1483730794.6273875
time period from 2017-01-07 02:26:34.627388 to 1483851967.4238088
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-01-08 12:06:07.423809 to 1483983439.6867409
time period from 2017-01-10 00:37:19.686741 to 1484127676.7567897
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-01-11 16:

time period from 2017-03-23 11:14:56.240442 to 1490302665.9071753
time period from 2017-03-24 03:57:45.907175 to 1490350878.825047
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
attempt: 000 | ['EService:Unavailable']
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-03-24 17:21:18.825047 to 1490394983.354596
time period from 2017-03-25 05:36:23.354596 to 1490446499.929683
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-03-25 19:54:59.929683 to 1490485994.0650702
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-03-26 06:53:14.065070 to 1490548424.5865216
time period from 2017-03-27 00:13:44.586522 to 1490617641.882824
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-03-27 19:27:21.882824 to 1490658102.7711666
time period from 2017-03-28 06:41:42.771167 to 1490710072.380

...

attempt: 000 | ['EService:Unavailable']
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-15 05:29:01.376415 to 1510732378.8277526
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-15 14:52:58.827753 to 1510762711.6659772
time period from 2017-11-15 23:18:31.665977 to 1510800748.685274
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-16 09:52:28.685274 to 1510834795.9450076
time period from 2017-11-16 19:19:55.945008 to 1510871293.925903
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-17 05:28:13.925903 to 1510915527.123963
time period from 2017-11-17 17:45:27.123963 to 1510956407.5508525
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-18 05:06:47.550853 to 1511017172.2376876
call rate limiter exceeded (counter=20, limit=20) 
 sleeping

time period from 2017-12-15 08:40:04.190114 to 1513335380.1861632
time period from 2017-12-15 17:56:20.186163 to 1513371803.6492853
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-16 04:03:23.649285 to 1513405693.914701
time period from 2017-12-16 13:28:13.914701 to 1513437519.465727
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-16 22:18:39.465727 to 1513469199.2988212
time period from 2017-12-17 07:06:39.298821 to 1513502667.1794724
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-17 16:24:27.179472 to 1513536245.5013657
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-18 01:44:05.501366 to 1513567345.5052073
attempt: 000 | ['EService:Unavailable']
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-18 10:22:25.505207 to 1513596078.1

time period from 2018-01-10 00:02:29.551322 to 1515546097.274664
time period from 2018-01-10 08:01:37.274664 to 1515574990.7547352
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-10 16:03:10.754735 to 1515604746.9313674
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-11 00:19:06.931367 to 1515633375.6293864
time period from 2018-01-11 08:16:15.629386 to 1515858838.512993
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-13 22:53:58.512993 to 1515886317.9526467
time period from 2018-01-14 06:31:57.952647 to 1515917402.4105196
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-14 15:10:02.410520 to 1515946463.342803
time period from 2018-01-14 23:14:23.342803 to 1515973664.0756946
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-15 06:47:

time period from 2018-02-06 06:59:06.601503 to 1517901673.6062417
time period from 2018-02-06 14:21:13.606242 to 1517927796.9465148
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-06 21:36:36.946515 to 1517954287.4490845
time period from 2018-02-07 04:58:07.449085 to 1517982550.2561674
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-07 12:49:10.256167 to 1518009670.0853868
time period from 2018-02-07 20:21:10.085387 to 1518037605.1652873
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-08 04:06:45.165287 to 1518067141.0695794
time period from 2018-02-08 12:19:01.069579 to 1518096841.8011653
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-08 20:34:01.801165 to 1518125796.2509775
time period from 2018-02-09 04:36:36.250978 to 1518153481.5025165
call rate limiter exceeded (counter=20, 

/Users/alastairong/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [69]:
K_BTC_EUR = K.get_training_data('XXBTZEUR')

time period from 2015-01-02 18:16:00.520070 to 1420286852.341405
time period from 2015-01-03 19:07:32.341405 to 1420324706.3725967
time period from 2015-01-04 05:38:26.372597 to 1420377687.4358373
time period from 2015-01-04 20:21:27.435837 to 1420437241.3787277
attempt: 000 | ['EService:Unavailable']
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-01-05 12:54:01.378728 to 1420482274.658581
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-01-06 01:24:34.658581 to 1420553253.230157
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-01-06 21:07:33.230157 to 1420604725.5882883
time period from 2015-01-07 11:25:25.588288 to 1420655491.2103224
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-01-08 01:31:31.210322 to 1420722419.5135658
time period from 2015-01-08 20:06:59.513566 to 1420805702.1885717
cal

time period from 2015-02-15 18:32:57.247468 to 1424033358.7778697
time period from 2015-02-16 03:49:18.777870 to 1424079276.7501493
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-02-16 16:34:36.750149 to 1424124772.3059528
time period from 2015-02-17 05:12:52.305953 to 1424177420.9204903
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-02-17 19:50:20.920490 to 1424251388.0328138
time period from 2015-02-18 16:23:08.032814 to 1424293347.5238564
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-02-19 04:02:27.523856 to 1424360283.5738547
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-02-19 22:38:03.573855 to 1424422518.159653
time period from 2015-02-20 15:55:18.159653 to 1424473435.3645911
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-02-21 06:0

time period from 2015-04-10 12:06:15.611949 to 1428690206.928618
time period from 2015-04-11 01:23:26.928618 to 1428821957.9437854
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-04-12 13:59:17.943785 to 1428917538.6640794
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-04-13 16:32:18.664079 to 1428954631.136825
time period from 2015-04-14 02:50:31.136825 to 1429003676.147862
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-04-14 16:27:56.147862 to 1429069550.8252664
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-04-15 10:45:50.825266 to 1429119124.5377905
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-04-16 00:32:04.537791 to 1429180734.5346348
time period from 2015-04-16 17:38:54.534635 to 1429263239.2756214
call rate limiter exceeded (counte

time period from 2015-06-09 14:25:03.061303 to 1433894681.303891
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-06-10 07:04:41.303891 to 1433953235.8668873
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-06-10 23:20:35.866887 to 1434037263.0259607
time period from 2015-06-11 22:41:03.025961 to 1434126966.4627516
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-06-12 23:36:06.462752 to 1434234009.811542
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-06-14 05:20:09.811542 to 1434310700.8405263
time period from 2015-06-15 02:38:20.840526 to 1434373453.8121877
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-06-15 20:04:13.812188 to 1434450692.1461298
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-06

time period from 2015-08-03 23:28:12.742070 to 1438683286.515884
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-08-04 17:14:46.515884 to 1438750670.7957678
time period from 2015-08-05 11:57:50.795768 to 1438799757.9276502
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-08-06 01:35:57.927650 to 1438862542.8154721
time period from 2015-08-06 19:02:22.815472 to 1438928814.671532
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-08-07 13:26:54.671532 to 1438967099.3306482
time period from 2015-08-08 00:04:59.330648 to 1439035367.6279674
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2015-08-08 19:02:47.627967 to 1439074210.3017058
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2015-08-09 05:50:10.301706 to 1439124127.0916967
time period from 2015-08-09 19:42

...

call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-26 23:19:07.326472 to 1511743557.826731
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-27 07:45:57.826731 to 1511771913.5648255
time period from 2017-11-27 15:38:33.564826 to 1511799656.892807
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-27 23:20:56.892807 to 1511828503.4337282
time period from 2017-11-28 07:21:43.433728 to 1511856424.693654
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-28 15:07:04.693654 to 1511884452.1730928
time period from 2017-11-28 22:54:12.173093 to 1511913076.449676
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-29 06:51:16.449676 to 1511940213.4806066
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-11-2

time period from 2017-12-20 17:32:05.698155 to 1513793012.9054527
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-21 01:03:32.905453 to 1513821335.5626569
time period from 2017-12-21 08:55:35.562657 to 1513848624.3748612
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-21 16:30:24.374861 to 1513874924.669532
time period from 2017-12-21 23:48:44.669532 to 1513902128.6962626
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-22 07:22:08.696263 to 1513927885.8146107
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-22 14:31:25.814611 to 1513953951.309128
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2017-12-22 21:45:51.309128 to 1513981423.9288151
time period from 2017-12-23 05:23:43.928815 to 1514009525.4312785
call rate limiter exceeded (count

time period from 2018-01-15 20:25:51.904005 to 1516049361.9239976
time period from 2018-01-16 03:49:21.923998 to 1516076637.4065392
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-16 11:23:57.406539 to 1516102340.367274
time period from 2018-01-16 18:32:20.367274 to 1516128507.287792
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-17 01:48:27.287792 to 1516155202.0352805
time period from 2018-01-17 09:13:22.035280 to 1516181697.1585457
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-17 16:34:57.158546 to 1516207549.3112168
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-17 23:45:49.311217 to 1516233639.112463
time period from 2018-01-18 07:00:39.112463 to 1516260091.9547117
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-01-18 14:21:

time period from 2018-02-06 22:38:04.546541 to 1517957612.8007903
time period from 2018-02-07 05:53:32.800790 to 1517984778.8948815
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-07 13:26:18.894881 to 1518010408.78896
time period from 2018-02-07 20:33:28.788960 to 1518036434.5438352
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-08 03:47:14.543835 to 1518064846.3897984
time period from 2018-02-08 11:40:46.389798 to 1518091098.7620971
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-08 18:58:18.762097 to 1518117661.5096622
time period from 2018-02-09 02:21:01.509662 to 1518145567.4599054
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-02-09 10:06:07.459905 to 1518171450.2611768
time period from 2018-02-09 17:17:30.261177 to 1518197504.3173137
call rate limiter exceeded (counter=20, li

/Users/alastairong/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [71]:
npdata = np.array(K_ETH_USD)
for ii in range(len(npdata)):
    #print(ii)
    npdata[ii].astype('float32')

In [72]:
with open('kraken_EUR_data2.pickle', 'wb') as f:
                pickle.dump((K_ETH_EUR, K_BTC_EUR, K_LTC_EUR), f)

In [73]:
with open('kraken_USD_data2.pickle', 'wb') as f:
                pickle.dump((K_ETH_USD, K_BTC_USD, K_LTC_USD), f)